In [2]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torchsummary import summary

# Basic Residual Block 
- ResNet 18, ResNet 34
- 논문에서 볼 수 있는 것과 같이 input x를 identity로 사용
- projection하는 과정에서 문제가 존재
    - stride가 1이 아니어서 feature map의 size가 다른 경우
    - input channel 수와 conv layer를 거친 output channel수가 다른 경우

In [3]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride = 1):
        super(BasicBlock, self).__init__()
        
        #stride를 통해 너비와 높이 조절
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size = 3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels*BasicBlock.expansion)
        )
        
        # identity mapping 
        self.shortcut = nn.Sequential()
        
        self.relu = nn.ReLU()
        
        # projection mapping using 1*1 conv
        # input과 conv layer를 거친 feature map size와 channel수가 다른 경우 맞추어주기 위해 추가
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )
            
        def forward(self, x):
            x = self.residual_function(x) + self.shortcut(x)
            x = self.relu(x)
            return x

# BottleNeck
- resnet 50,101, 152

In [4]:
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride = 1):
        super().__init__()
        
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels* BottleNeck.expansion),
            nn.ReLU(),
        )
        self.shortcut = nn.Sequential()
        
        self.relu = nn.ReLU()
        
        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion)
            )
            
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

# ResNet 구현

In [5]:
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes = 10, init_weights = True):
        super().__init__()
        
        self.in_channels = 64
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # conv2_x는 max_pool layer를 앞에서 거치기 때문에 stride가 2인 경우 x
        # conv3_x부터는 첫 번째 conv layer에서 feature map size를 줄여주기 때문에 2로 설정
        self.conv2_x = self.make_layer(block, 64, num_block[0], 1) # output channel 64
        self.conv3_x = self.make_layer(block, 128, num_block[1], 2) # output channel 128
        self.conv4_x = self.make_layer(block, 256, num_block[2], 2) # output channel 256
        self.conv5_x = self.make_layer(block, 512, num_block[3], 2) # output channel 512
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        if init_weights:
            self._initialize_weights()
        
    # 각 block의 첫번째 conv layer에서만 stride=2로 지정하여 feature map의 크기를 반으로 줄여준다.
    # 주의 : 정수의 합이 아닌 리스트의 합
    def make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1) 
        # ex) [1, 1], [2, 1, 1], [2, 1, 1, 1]....
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            # 하나의 layer가 끝나게 되면 channel수를 expansion의 값에 따라서 증가시켜준다.
            self.in_channels = out_channels * block.expansion
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x
     
    # weight 초기화 함수 정의 
    def _initialize_weights(self):
        #모델의 모듈을 차례대로 불러온다.
        for m in self.modules():
            #isinstance() -> 차례로 layer을 입력하여, layer의 형태를 반환(nn.Conv2d, nn.BatchNorm2d ...)
            if isinstance(m,nn.Conv2d):
                # Kaming Initialization
                # 모듈의 가중치를 kaming he normal로 초기화합니다.
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm2d):
                #torch.nn.init.constant_(tensor, val) -> tensor을 val로 초기화
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)

            elif isinstance(m, nn.Linear):
                #torch.nn.init.normal_(tensor, mean=0.0, std=1.0) -> tensor을 mean, std의 normal distrubution으로 초기화
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0) 
        
# block의 종류, num_block 입력
def resnet18():
    return ResNet(BasicBlock,[2,2,2,2])

def resnet34():
    return ResNet(BasicBlock,[3,4,6,3])

def resnet50():
    return ResNet(BottleNeck,[3,4,6,3])

def resnet101():
    return ResNet(BottleNeck,[3,4,23,3])

def resnet152():
    return ResNet(BottleNeck,[3,8,36,3])
        

In [6]:
#네트워크 생성
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = resnet50().to(device)
x = torch.randn(3, 3, 224, 224).to(device)
output = model(x)
print(output.size())

cuda:0
torch.Size([3, 10])


In [7]:
summary(model, (3, 256, 256), device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]           4,096
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
           Conv2d-11          [-1, 256, 64, 64]          16,384
      BatchNorm2d-12          [-1, 256, 64, 64]             512
             ReLU-13          [-1, 256, 64, 64]               0
           Conv2d-14          [-1, 256,